In [24]:
import codecs
import re
import os

In [3]:
# from https://segmentfault.com/q/1010000000732038
# to detect english character ONLY
def isAlpha(word):
    try:
        return word.encode('ascii').isalpha()
    except UnicodeEncodeError:
        return False

In [84]:
# from http://www.cnblogs.com/kaituorensheng/p/3554571.html
# -*- coding: cp936 -*-
def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:                              #全角空格直接转换            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248

        rstring += chr(inside_code)
    return rstring

In [20]:
filename_cmu = "cmudict-0.7b"

d_cmu = dict()
max_num = 0 # max number of (x) <-- a word with multiple pronunciations
f = open(filename_cmu,"r")
for line in f.readlines():
    if line[0:3] != ";;;":
        line = line[:-1]
        line = line.split("  ",1)
        d_cmu[line[0]] = line[1]
        if line[0][-1] == ")":
            try: 
                num_cur = int(line[0][-2])      
                if int(line[0][-2]) > max_num:
                    max_num = num_cur
            except:
                continue

print("Finish loading cmu dictionary")
print("Number of English lexicon: " + str(len(d_cmu)))

Finish loading cmu dictionary
Number of English lexicon: 133854


In [18]:
filename_th = "thchs30-lexicon.txt"

d_th = dict()
with codecs.open(filename_th, 'r', 'utf-8') as f:
    for line in f.readlines():
        line = line.split()
        d_th[line[0]] = ' '.join(line[1:])
print("Finish loading thchs30 dictionary")
print("Number of Chinese lexicon: " + str(len(d_th)))

Finish loading thchs30 dictionary
Number of Chinese lexicon: 8873


In [16]:
# the length of thchs30-lexiconp.txt is the same to thchs30-lexicon.txt
# and the content looks same..
'''
filename_thp = "thchs30-lexiconp.txt"

d_thp = dict()
max_num = 0
#f = open(filename_th,"r")
with codecs.open(filename_th, 'r', 'utf-8') as f:
    for line in f.readlines():
        line = line.split()
        d_thp[line[0]] = ' '.join(line[1:])
print(len(d_thp))
print("Finish loading thchs30 dictionary p")
'''

'\nfilename_thp = "thchs30-lexiconp.txt"\n\nd_thp = dict()\nmax_num = 0\n#f = open(filename_th,"r")\nwith codecs.open(filename_th, \'r\', \'utf-8\') as f:\n    for line in f.readlines():\n        line = line.split()\n        d_thp[line[0]] = \' \'.join(line[1:])\nprint(len(d_thp))\nprint("Finish loading thchs30 dictionary p")\n'

In [182]:
dir = "./transcript"

# to get the full word list
filenames = os.listdir(dir)
words = []
for filename in filenames:
    if filename[-4:] == ".txt":
        with codecs.open(dir+"/"+filename, 'r', 'utf-8') as f:
            for line in f.readlines():
                sentence = line.split() # 1st element -- audio file id, 2nd element -- start time, 3rd element -- end time
                words_cur = sentence[4:]   
                words += words_cur
            f.close()

print("Finish concatenating words")
print("Number of words: " + str(len(words)))
print("Number of unique words: " + str(len(set(words))))

Finish concatenating words
Number of words: 518489
Number of unique words: 28610


In [187]:
# create a dictionary of waste words
words_waste = []
words_1 = [] # [x]
words_2 = [] # (x)
        
for word in words:
    if word[-1] == "]" or word[0] == "[":
        word = re.findall(r'''(\[.+?\])''',word)
        words_1 += word
    elif word[-1] == ")" or word[0] == "(":
        word = re.findall(r'''(\(.+?\))''',word)
        words_2 += word
        
words_waste = words_1 + words_2

d_waste = dict()
for word in words_waste:
    try:
        d_waste[word] += 1
    except:
        d_waste[word] = 1

d_waste2idx = dict()
idx = -1
for k in d_waste.keys():
    idx += 1
    d_waste2idx[k] = 'SIL' + str(idx)

d_idx2waste = {v:k for k,v in d_waste2idx.items()}

print("Finish creating waste word dictionary")
print("Number of waste word type: " + str(len(set(words_waste))))

Finish creating waste word dictionary
Number of waste word type: 168


In [188]:
# fix and output the transcript text going to be used
dir = "./transcript/"
dir_save = "./transcript_fixed/" 
filenames = os.listdir(dir)
words_all = []
text_all = []
text_unparse = []
for filename in filenames:
    if filename[-4:] == ".txt":
        with codecs.open(dir+filename, 'r', 'utf-8') as f:
            text = []

            for line in f.readlines():
                sentence_origin = line.strip()
                
                words_mm_cur = set(re.findall(r'''o(\[.+?\])''',line))
                if len(words_mm_cur) > 0:
                    sentence_origin = sentence_origin.replace('o[mm]','omm')
                
                # replace waste words with SILx; space after ] or before [ to avoid situations like co[mm]on or co[mm]unication
                words_waste_cur = set(re.findall(r'''(\[.+?\])''',line) + re.findall(r'''(\(.+?\))''',line))
                for element in words_waste_cur:
                    try:
                        sentence_origin = sentence_origin.replace(element,' ' + d_waste2idx[element] + ' ')
                    except:
                        continue
                
                # get rid of "#"
                sentence_origin = sentence_origin.replace('#',' ')
                
                # get rid of "="
                sentence_origin = sentence_origin.replace('=',' ')
                
                # deal with %word% like %chelsia%
                sentence_origin = sentence_origin.replace('%',' ')
                
                # deal with "word" like "william"
                sentence_origin = sentence_origin.replace('"',' ')
                
                # deal with pronunciation of single letter, for which the trascript is like P. S., I. T., etc.
                sentence_origin = sentence_origin.replace('.',' ')
                
                # Q2B
                sentence_chars = list(sentence_origin)
                for char_idx in range(len(sentence_chars)):
                    sentence_chars[char_idx] = strQ2B(sentence_chars[char_idx])
                sentence_origin = "".join(sentence_chars)
                
                # trim again incase the substitution brings in space
                sentence_origin = sentence_origin.strip()
                              
                sentence = sentence_origin.split() # 1st element -- audio file id, 2nd element -- start time, 3rd element -- end time
                info_cur = sentence[:3] # idx: 0~2
                words_cur = sentence[3:] # idx: 3~
                unparse = False
    
                for word in words_cur: # filter out concatenated characters having Chinese characters, with length over 4
          
                    # filter out unparsed element (unparse = F -> unparse = T)
                    # for example, [leh]每次上学的时候daddy都会买糕点给我吃买马来糕#kuih# (idx: 53628)
                    try:
                        d_cmu[word.upper()] # if the word is a recorded English word
                        continue
                    except:
                        try:
                            d_idx2waste[word] # if the word is SILx
                            continue
                        except:
                            try:
                                d_th[word] # if the word is a recorded Chinese word
                                continue
                            except:
                                word_split = word.split("-")
                                if len(word_split) > 1: # pass if it is in the form of "a-b"
                                    continue
                                elif isAlpha(word.replace("'","")): # words like o'clock
                                    continue
                                else:
                                    if len(word) > 4 and isAlpha(word) == False:
                                        unparse = True
                                        break

                if unparse:
                    text_unparse += [sentence_origin]
                else:
                    sentence_cur = " ".join(info_cur) + " " + " ".join(words_cur)
                    text += [sentence_cur]
                    text_all += [sentence_cur]
                    words_all += words_cur
    
            f.close()
            
        #save files without unparsed lines
        with codecs.open(dir_save+filename, 'w', 'utf-8') as f:
            for line in text:
                f.write(line + "\n")
        f.close()

with codecs.open(dir_save+'unparsed.txt', 'w', 'utf-8') as f:
    for line in text_unparse:
        f.write(line + "\n")
    f.close()

with codecs.open(dir_save+'text.txt', 'w', 'utf-8') as f:
    for line in text_all:
        f.write(line + "\n")
    f.close()
            
print("Finish reading lines")
print("Number of segmented utterance: "+str(len(text_all)))
print("Number of unsegmented utterance: "+str(len(text_unparse)))

Finish reading lines
Number of segmented utterance: 45872
Number of unsegmented utterance: 9057


In [189]:
# output silence_phones.txt
with codecs.open(dir_save+'silence_phones.txt', 'w', 'utf-8') as f:
    f.write('SIL\n')
    for k,v in d_waste2idx.items():
        f.write(v + "\n")
f.close()
print("Finish writing silence_phones.txt")

# output nonsilence_phones.txt
phones = []

with codecs.open('cmudict-0.7b.phones', 'r', 'utf-8') as f:
    for phone in f.readlines():
        phone = phone.split()
        phones += [phone[0]]
f.close()

with codecs.open('thchs30-nonsilence_phones.txt', 'r', 'utf-8') as f:
    for phone in f.readlines():
        phones += [phone[:-1]] #negelect \n
f.close()

with codecs.open(dir_save+'nonsilence_phones.txt', 'w', 'utf-8') as f:
    for phone in phones:
        f.write(phone + "\n")
f.close()

print("Finish writing nonsilence_phones.txt")
#print(phones)

Finish writing silence_phones.txt
Finish writing nonsilence_phones.txt


In [191]:
words_all = [word.upper() for word in words_all]
words_all_uniq = list(set(words_all))
words_oov = []
d_train = dict()
for word in words_all_uniq:
    try:
        d_train[word] = d_cmu[word]
        for i in range(1,max_num+1):
            try:
                d_train[word+"(i)"] = d[word+"(i)"]
            except:
                break
    except:
        try:
            d_train[word] = d_th[word]
        except:
            words_oov += [word]
            continue
print("Finish lexicon filtering")
print("Number of words used: " + str(len(words_all)))
print("Number of unique words used: " + str(len(words_all_uniq)))
print("Number of unique words in filtered lexicon file: " + str(len(d_train)))

Finish lexicon filtering
Number of words used: 534296
Number of unique words used: 17531
Number of unique words in filtered lexicon file: 11205


In [192]:
# output lexicon.txt
lexicons = [str(k) + " " + str(v) + "\n" for k,v in d_train.items()]

with codecs.open(dir_save+'lexicon.txt', 'w', 'utf-8') as f:
    f.write('<oov> <oov>\n')
    for lexicon in lexicons:
        f.write(lexicon)
    #for word in words_oov:
    #    f.write(str(word) + " " + "<oov>\n")
    for k,v in d_waste2idx.items():
        f.write(v + " sil" + "\n")
        
f.close()

print("Finish writing lexicon.txt")

Finish writing lexicon.txt


In [262]:
text[32788] # cannot be understood by human

'NI61FBP_0101\t696201\t702891\t有  以前  我 sec one sec two 就 用 里 谷 join 然后 sec three sec four 有 join 那个 robocup\n'

In [263]:
text[19959] # gibberlish

'NI22FBP_0101\t668052\t669342\t他  对  她  有  ·䞠\x84\x9f枠§\x89\n'

In [264]:
text[20115] # loss of label: "#angpow" should be "#angpow#"

'NI22FBP_0101\t1040195\t1044030\tthen 他  会  给 ten dollar #angpow ha ha ha\n'

In [265]:
text[46226] # space between labeled term: if we use the default split method, #kim gary# will be split into 2 words
            # as the number of space between is 1 between english words and 2 between chinese words, it's really troublesome 
            # to do right split to avoid this problem

'UI17FAZ_0103\t751720\t754727\tah sorry我不吃#kim gary#的 呵呵呵\n'

In [266]:
text[21298] # % indicates a special noun

'NI24MBP_0101\t1138380\t1140790\t我  我  不是  去 %instep% 我是  去 g. i. p.\n'

In [267]:
text[21373] # % indicates a special noun

'NI25MBQ_0101\t583590\t587390\t所以 %zhi=hong% %zhi=hong% 有点 %zhi=hong% was very stress because er\n'

In [268]:
text[21503] # % indicates a special noun

'NI26FBP_0101\t1073610\t1076900\tit was then %sweety% decided that 他  会 take up cultural\n'

In [269]:
text[21513] # % indicates a special noun

'NI26FBP_0101\t1261440\t1262820\t回来  继续  玩 %viwawa%\n'

In [270]:
text[27698] # inconsistent annotation: according to the codebook, "sashimi" shoud be annotated as "#sashimi#" 
            # because it's a japanese word

'NI43FBP_0101\t1494840\t1501690\t不错  蛮  很好  吃  的  那边  的 %sashimi% 是  很  新鲜  的  那一  种  啊  就  一直  吃  "sashimi"\n'

In [271]:
text[48495] # wrongly split a word in transcription: "i. t." should be "IT", "o. k." should be "ok" in our sense 
            # it may help if we discard the "." after the character

'UI21MAZ_0101\t689491\t700786\t可能出来我发现我是不喜欢 i. t. 的 那时候如果现在是我工作我又有钱赚然后有可以得到经验如果又知道是 o. k. 的话到时半工读啰会比较好啦[mmm] \n'

In [272]:
text[54294] # need to check the audio to see if "epiko" is pronunced as a word or letters

'UI29FAZ_0102\t64704\t71545\t又有m.m.lah 我的我的有的有的我都不懂啊知道最著名的是e.p.i.k.o. [ah] h.r. [ah] err\n'

In [273]:
text[53626] # wrong transcription of mandarin: "大嫂干净" should be "打扫干净"

'UI28FAZ_0101\t221989\t230002\t大嫂干净[mmm] move 去那个新#taman#之后新的garden\n'

In [274]:
text[53390] # wrong transcription of english: "netbook" should be "notebook"

'UI27FAZ_0103\t1112733\t1119724\t啊touch screen的 and 通常会有人会买i. pad compared to netbook\n'

In [275]:
text[45496] # wrong annotation: "?" might refer to a non-transcribable sound?

'UI16MAZ_0103\t747433\t752878\t哦那个我还不能啦 但是我？了起来大约大约想想这样才可以啦 \n'

In [276]:
text[52996] # for this kind of unparsed utterance, we can split chinese charater and english words

'UI27FAZ_0101\t752877\t756798\tw. h. o.是world world health organize organization\n'

In [161]:
i = 0
for k in d_th.keys():
    if len(k) > 4:
        print(k)
        i+=1
print(i)

埃塞俄比亚
多斯桑托斯
<SPOKEN_NOISE>
人民大会堂
一九九二年
五千四百五十
聪明反被聪明误
乌兹别克斯坦
百分之二十
四百二十三
塞尔维亚人
原因很简单
中华人民共和国
幸福的家庭
中国科学院
一千二百二十七
叫我怎么办
二百九十一
三思而后行
艾森豪威尔
四千五百七十
二百五十五
一百七十四
三百五十三
自己的房间
一百三十三
新概念英语
阿拉伯数字
二十一世纪
一百三十四
三千四百五十
五万五千五百五十五
一百三十五
33
